In [1]:
import numpy as np
import pandas as pd
from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr

In [3]:
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [33]:
ds = load_dataset("beanham/spatial_join_dataset")
test=ds['test']
gt=np.array(test['label'])

## prepare configs
models = ['llama3', '4o_mini', 'gemini', 'qwen']
methods = ['zero_shot', 'few_shot'] 
modes = ['no_heur', 'with_heur_hint', 'with_heur_value']
heuristics = ['angle', 'distance', 'comb']
configs=['_'.join(i) for i in list(product(methods, modes, heuristics))]
configs.remove('zero_shot_no_heur_angle')
configs.remove('zero_shot_no_heur_distance')
configs.remove('zero_shot_no_heur_comb')
configs.remove('few_shot_no_heur_angle')
configs.remove('few_shot_no_heur_distance')
configs.remove('few_shot_no_heur_comb')
configs.insert(0, 'zero_shot_no_heur')
configs.insert(7, 'few_shot_no_heur')

## evaluate on a subset
np.random.seed(100)
index=np.random.randint(0, 3069, 1000)

### Heuristics

In [7]:
## heuristcis
for i in [1,2,5,10,20]:
    pred=np.array(test['min_angle'])<=i
    metrics=metric_calculation(pred, gt)
    print(i, metrics[0], metrics[1])

1 0.867 0.8649569334461061
2 0.908 0.9047760996290408
5 0.946 0.9429281655844156
10 0.948 0.9445921985815603
20 0.934 0.9288927961938254


In [8]:
## heuristcis
for i in [1,2,3,4,5]:
    pred=np.array(test['min_euc_dist'])>=i
    metrics=metric_calculation(pred, gt)
    print(i, metrics[0], metrics[1])

1 0.838 0.8138786764705882
2 0.847 0.8255016258002119
3 0.851 0.8311329618258494
4 0.852 0.8326549072817729
5 0.835 0.8173321864008795


In [10]:
angles=[1,2,5,10,20]
distances=[1,2,3,4,5]
test_gt=np.array(test['label'])
combs=list(product(angles, distances))
accuracy=[]
for a,d in combs:
    pred=(np.array(test['min_euc_dist'])>=d)&(np.array(test['min_angle'])<=a)
    accuracy.append([a,d,accuracy_score(test_gt, pred)])
accuracy=pd.DataFrame(accuracy,columns=['angle','distance','acc'])
accuracy

,angle,distance,acc
0,1,1,0.868
1,1,2,0.868
2,1,3,0.866
3,1,4,0.865
4,1,5,0.847
5,2,1,0.910
6,2,2,0.910
7,2,3,0.908
8,2,4,0.907
9,2,5,0.885


### LLMs

In [38]:
results=[]
for model in models:
    print(f'Model: {model}...')
    for config in configs:
        pred=np.load(f'base/{model}/{model}_{config}.npy')
        pred=post_processing(pred)        
        metrics=metric_calculation(pred[index], gt)
        results.append([config, model, metrics[0], metrics[1]])
results=pd.DataFrame(results, columns=['config', 'model', 'acc', 'f1'])

Model: llama3...
Model: 4o_mini...
Model: gemini...
Model: qwen...
